This example shows how to create a line emission model. It uses a galaxy cluster from a Gadget SPH cosmological dataset, and will create a thermal model out of the gas particles and will use the dark matter particles to add line emission to the spectrum, assuming that the emission comes from some decay process of the dark matter.

First, we load the modules:

In [ ]:
%matplotlib inline
import matplotlib
matplotlib.rc("font", size=18, family="serif")
import yt
import numpy as np
import matplotlib.pyplot as plt
from yt.units import mp
import pyxsim

and we load the dataset in yt:

In [ ]:
ds = yt.load("snapshot_033/snap_033.0.hdf5")

To get a sense of what the cluster looks like, let's take a slice through the density and temperature:

In [ ]:
slc = yt.SlicePlot(ds, "z", ["density","temperature"], center="max", width=(3.,"Mpc"))
slc.show()

Now set up a sphere centered on the maximum density in the dataset:

In [ ]:
sp = ds.sphere("max", (1.0, "Mpc"))

and create a thermal source model:

In [ ]:
spec_model = pyxsim.TableApecModel(0.05, 11.0, 10000)
thermal_model = pyxsim.ThermalSourceModel(spec_model, Zmet=0.3)

Now we'll set up a line emission field for the dark matter particles. We won't try to replicate a specific model, but will simply assume that the emission is proportional to the dark matter mass. Note that this field is a particle field.

In [ ]:
norm = yt.YTQuantity(100.0, "g**-1*s**-1")

def _dm_emission(field, data):
    return norm*data["PartType1", "particle_mass"]
ds.add_field(("PartType1","dm_emission"), function=_dm_emission, particle_type=True, 
             units="photons/s")

Now we can set up the `LineSourceModel` object. The first argument is the line center energy in keV, and the second is the field we just created, that sets up the line amplitude. There is another parameter, `sigma`, for adding in broadening of the line, but in this case we'll rely on the velocities of the dark matter particles themselves to produce the line broadening.

In [ ]:
line_model = pyxsim.LineSourceModel(3.5, ("PartType1", "dm_emission"))

Now set up the parameters for generating the photons:

In [ ]:
exp_time = (300., "ks") # exposure time
area = (1000.0, "cm**2") # collecting area
redshift = 0.05

and actually generate the photons:

In [ ]:
thermal_photons = pyxsim.PhotonList.from_data_source(sp, redshift, area, exp_time, thermal_model)
line_photons = pyxsim.PhotonList.from_data_source(sp, redshift, area, exp_time, line_model)

photons = line_photons + thermal_photons

Next, set up an absorption model and project the photons for the total set and the line set by itself:

In [ ]:
thermal_photons.parameters

In [ ]:
line_photons.parameters

In [ ]:
nH_sim = 0.02
abs_model = pyxsim.TableAbsorbModel("tbabs_table.h5", nH_sim)

events = photons.project_photons("y", absorb_model=abs_model)
line_events = line_photons.project_photons("y", absorb_model=abs_model)

Write the raw, unconvolved spectra to disk:

In [ ]:
events.write_spectrum("all_spec.fits", bin_type="energy", emin=0.1, emax=10.0, nchan=5000, clobber=True)
line_events.write_spectrum("line_spec.fits", bin_type="energy", emin=0.1, emax=10.0, nchan=5000, clobber=True)

Now let's plot up both spectra. We see that we have a thermal spectrum with the addition of a line at 3.5 keV (in real life such a line would not be so prominent, but it makes the example easier to see):

In [ ]:
import astropy.io.fits as pyfits
f1 = pyfits.open("all_spec.fits")
f2 = pyfits.open("line_spec.fits")
fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(111)
ax.loglog(f1["SPECTRUM"].data["ENERGY"], f1["SPECTRUM"].data["COUNTS"])
ax.loglog(f2["SPECTRUM"].data["ENERGY"], f2["SPECTRUM"].data["COUNTS"])
ax.set_xlim(0.1, 10)
ax.set_ylim(1, 3.0e4)
ax.set_xlabel("E (keV)")
ax.set_ylabel("counts/bin")

Let's zoom into the region surrounding the line, seeing that it has some broadening due to the random velocities of the dark matter particles:

In [ ]:
ax.set_xscale("linear")
ax.set_xlim(3,3.7)
ax.set_ylim(1.,3.0e2)
fig